# Lane-lines detection

## Intro

### Goal

- By the end of the day you will write an algorithm to delect and draw lanelines.


### Milestones

1. Write the algorithm and tune the params.
2. Test it on different images.
3. Load and process video frame by frame.

### Algorithm Defenition

What do you think the steps are ?

### Algorithm Steps

1. Load an image.
2. Extract lane lines.
    - Threshold image.
    - Detect edges(Canny edge detection).
    - Extract lines(Hough lines).
    - Find lines equations.
3. Draw lanelines.

### Input/Output

Input:
- Image.
- Video (Frames).

Output:
- Annotated image with lines.

In [ ]:
from IPython.display import HTML

In [ ]:
input_clip_name = "test_videos/solidWhiteRight.mp4"

HTML(f"""
<video width="960" height="540" controls>
  <source src="{input_clip_name}">
</video>
""")

In [ ]:
output_clip_name = "test_videos_output/solidWhiteRight_good.mp4"

HTML(f"""
<video width="960" height="540" controls>
  <source src="{output_clip_name}">
</video>
""")

## Process ...

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

: 

### Load image

In [ ]:
img = cv2.imread('test_images/solidYellowLeft.jpg')
print('Shape: ', img.shape)
plt.imshow(img[:,:,::-1]);

### Convert to grey

In [ ]:
def toGray(img):
  return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray = toGray(img)
plt.imshow(gray, cmap='gray');

### Detect edges

#### What is canny edge detection?

- A method used to detect edges in an image while suppressing noise.
- Steps:
  - Grayscale Conversion
  - Gaussian Blur
  - Determine the Intensity Gradients
  - Non Maximum Suppression
  - Double Thresholding
  - Edge Tracking by Hysteresis
  - Cleaning Up

#### Resources:

- https://www.justin-liang.com/tutorials/canny/
- https://homepages.inf.ed.ac.uk/rbf/HIPR2/canny.htm
- https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html
- https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga2a671611e104c093843d7b7fc46d24af

In [ ]:
low_th = 40
high_th = 100
def canny(img):
  blur = cv2.GaussianBlur(img, (7,7), 0)
  return cv2.Canny(blur, low_th, high_th)

edges = canny(gray)
plt.imshow(edges, cmap='gray');

### ROI

Remove unwanted region to focus on the region we are interested in.

In [ ]:
def maskROI(img):
  mask = np.zeros_like(img)
  if len(img.shape) > 2:
    channel_count = img.shape[2]
    ignore_mask_color = (255,) * channel_count
  else:
    ignore_mask_color = 255
  imshape = img.shape        
  vertices = np.array([
    [
      ((1/6*imshape[1]), imshape[0]),
      ((5/12*imshape[1]), (3/5*imshape[0])),
      ((7/12*imshape[1]), (3/5*imshape[0])),
      ((9/10*imshape[1]), imshape[0])
    ]], dtype=np.int32)
  cv2.fillPoly(mask, vertices, ignore_mask_color)
  masked_edges = cv2.bitwise_and(img, mask)
  return(masked_edges)

masked = maskROI(edges)
plt.imshow(masked, cmap='gray');

In [ ]:
test = np.ones((500, 1000))*100
test[-1, 500] = 255
plt.imshow(maskROI(test), cmap='gray');

### Hough Lines

An alg. to detect lines in an binary image.

In [ ]:


def hough_lines(img):
  RHO = 1                 # try: 1 - 4 (0.5 increments)  
  THETA = np.pi/180       # Usually this is Ok
  MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
  MIN_LINE_LEN = 5 
  MAX_LINE_GAP = 50 
  lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
  return lines
  
lines = hough_lines(masked)
print (lines.shape)
print(lines)

In [ ]:
def draw_lines(lines, masked_edges):
  color = [243, 105, 14]
  thickness = 12
  lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
  for line in lines:
    for x1,y1,x2,y2 in line:
      cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
  return(lines_image)

lined_img = draw_lines(lines, masked)
plt.imshow(lined_img);

### Formulate and Draw Lanes

In [ ]:
from helpers import helpers_formulate_lanes as formulate_lanes
lanes = formulate_lanes(lines, masked)
print(lanes)
lanes_img = draw_lines(lanes, img)
plt.imshow(lanes_img);

In [ ]:
final_image = cv2.addWeighted(img[:,:,::-1], 0.9, lanes_img, 1, 0)

In [ ]:
plt.imshow(final_image);